<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Linux Introduction: Networking</h1>  

©2000-24 Chris Seddon 
</div>

Let's start by checking which machine we are on:

In [ ]:
hostname

That tells us our fully qualified hostname.  It includes our domain name.  

A domain name is an identification string that defines a realm of control within the Internet. Domain names are used in various networking senarios.

If we are just interested in the hostname without the domain:

In [ ]:
hostname -s

Devices on the Internet are assigned a unique IP address.  We can use the "ifconfig" command to check on our IP address, but it usually returns too much information.  The following command filters out just the IP information:

In [ ]:
ifconfig | grep inet

The above command is reporting both IP version 4 (inet) and IP version version 6 (inet6) addresses.  Theses are usually abreviated to IPv4 and IPv6.

IPv4 uses 32 bit addresses and was the original procol used on the internet.  However with the rapid growth of the Internet after 1995, it became evident that we would run out of addresses in the early 21st century.  This prompted the development of IPv6.  IPv6 uses 128-bit addresses, theoretically allowing $2^{128}$, or approximately $3.4×10^{38}$ addresses. The actual number is slightly smaller, as multiple ranges are reserved.

Nowadays it is common to use both IPv4 and IPv6 addresses.  In time IPv4 will be phased out.

My output from the above command was:
<pre>
inet 172.23.159.7  netmask 255.255.255.0  broadcast 172.23.159.255
inet6 fe80::a94:efff:fe5b:c1bc  prefixlen 64  scopeid 0x20<link>
inet 127.0.0.1  netmask 255.0.0.0
inet6 ::1  prefixlen 128  scopeid 0x10<host>
</pre>
This shows I have 2 IPv4 addresses:
* 172.23.159.7
* 127.0.0.1  

IPv4 addresses are split into 4 octets and displayed in decimal.  The first address is my internet address and the second one is called the loopback address.  It is always `127.0.0.1` and is used for testing network programs locally and simulating the real internet.

IPv6 addresses more complicated because they are 128 bits long.  They are normally displayed in hex:
* fe80::a94:efff:fe5b:c1bc
* ::1  

but with leading zeros omitted and empty octets displayed as `::`.  Again, the second address is the loopback.  If we write the IPv6 addresses in full (with the zeros included) they are:

* fe80:0000:0000:0000:0a94:efff:fe5b:c1bc
* 0000:0000:0000:0000:0000:0000:0000:0001  

Note that "ifconfig" reports a lot more information:

In [ ]:
ifconfig

The above shows each network interface amonst other things whether it is UP or DOWN and error counts.  Full information can be found in the manual pages:

In [ ]:
man ifconfig

There are usually several machines within a domain.  There are several ways to discover such machines, but we will use the "arp" command to see what's out there.

"arp" is the Address Resolution Protocol, which is used to find the address of a network neighbour for a given hostname:

In [ ]:
arp -a

The address shown above is the MAC address of the network card.  The MAC address is unique to the hardware and is nothing to do with the IP address.

We can run "arp" again to show IPv4 addresses instead of hostnames:

In [ ]:
arp -n

We can use the "ping" command to see if a host is running.  Normally we don't supply any options an "ping" runs until we kill it.  I want it to finish quickly so I've given it a count of 5 pings:

In [ ]:
ping -c 5 cs05r-sc-serv-04.diamond.ac.uk

Since we are contacting a host on our own network the pings are very quick.  Let's try pinging the IBM network:

In [ ]:
ping -c 5 www.ibm.com

Well that took a little longer.  

We are on the domain<pre>diamond.co.uk</pre>
We can use "nslookup" to obtain information about our domain:

In [ ]:
nslookup diamond.ac.uk

Often we want to obtain information about a program running on our own machine.  So let's run a Python server on port 8147 and use network commands to check that it is running:

In [ ]:
python -m http.server 8147 &

An IP address of `0.0.0.0` is a wildcard to mean any local IP address to this machine.  

We can use "netstat" to find details of our server.  

The options are:

* v : verbose
* a : all sockets
* n : use numerical addresses

In [ ]:
netstat -van 2>/dev/null | grep ':8147'

Officially the "netstat" command is obsolete and we should be using "ss",  This has the advantage of working for both IPv4 and IPv6.  Furthermore it tells us the PID in case we want to kill the server.  

The options used are:

* t : TCP sockets
* u : UDP sockets
* l : listening sockets
* p : show process using socket
* n : use numerical addresses

In [ ]:
ss -tulpn | grep ':8147'

Yet another way to see this information is to use "lsof".  The "-i" stands for internet files and the colon introduces the port number:

In [ ]:
lsof -i :8147

Finally let's kill off our server with the "fuser" command.  "fuser" will display the PID of the process using a particular port.  However, if we use the "-k" option, "fuser" will kill the process:

In [ ]:
fuser -k 8147/tcp
jobs

Another interesting network command is "mtr".  "mtr" combines the functionality of the traceroute and ping programs in a single network diagnostic tool.

The options used are:

* c : count of number of pings
* r : produce a report
* w : wide format (don't truncate hostnames)

In [ ]:
mtr -c 5 -r -w www.bbc.co.uk

We complete this tutorial by looking `/etc/services` which contains a list of standard Unix services and the port they run on.  Note that services can run using TCP or UDP or both.  Not all these services will actually be run on any particular host.

In [ ]:
cat /etc/services

Some of the more important services from this file are filtered out below:

In [ ]:
egrep ' (21|22|80|443)[/]' /etc/services

Note that port 80 and 443 are the standard ports for internet http and https trafic.  The file transfer program, "ftp" runs on port 21.

The secure shell, "ssh" runs on port 22.  "ssh" allows us to login to a remote machine and then run commands on the said host.

"ssh" uses authentication before it allows a connection.  To ensure it is safe to connect to the remote host, "ssh" uses an ECDSA key fingerprint of the remote host (but stored on the local machine).  Sometimes this needs to be reset with the "-R" option, e.g.:
<pre>ssh-keygen -R cs05r-sc-serv-10.diamond.ac.uk</pre>

Unfortunately, Jupyter notebook's bash shell doesn't read standard input, which makes it impossible to enter your password when "ssh" requests it.  Furthermore, if you connect to Diamond with "NoMachine", security at Diamond will restrict which machines you can "ssh".

However, once we are on the Diamond network we can `ssh` to one of the training machines:
<pre>
diamtr501
diamtr502
diamtr503
diamtr504
diamtr505
diamtr506
diamtr507
diamtr508
diamtr509
diamtr510
</pre>
We can try this out by creating an `xterm` and performing a `ssh` from there.  

Type the following in the `xterm`:
<pre>ssh diamtr501</pre>
Try a different machine if you like.

In [ ]:
xterm -fg black -bg white -fa 'Monospace' -fs 14 &